### LIBRARIES

In [1]:
# LIBRARIES
import math
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import chime        # Notification sounds
from matplotlib import rc
from tensorflow import keras
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split, ShuffleSplit, GroupShuffleSplit
from sklearn import preprocessing
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from functools import partial
%load_ext chime

### SETTING STYLE

In [2]:
# STYLE
# Plot/table options
desired_width = 400
pd.set_option('display.width', desired_width)
pd.set_option('display.max_columns', 12)
rc('font', **{'family': 'serif', 'serif': ['Times New Roman']})
plt.rcParams['font.size'] = 14

# Colors RGB
ccm_black_rgb = [0, 0, 0]
ccm_dgray_rgb = [85, 85, 85]
ccm_gray_rgb = [210, 210, 210]
ccm_dblue_rgb = [25, 50, 120]
ccm_blue_rgb = [20, 80, 200]
ccm_lblue_rgb = [170, 200, 230]
ccm_red_rgb = [190, 0, 0]
ccm_orange_rgb = [255, 100, 0]

colors = [ccm_black_rgb, ccm_dgray_rgb, ccm_gray_rgb, ccm_dblue_rgb, ccm_blue_rgb, ccm_lblue_rgb, ccm_red_rgb,
          ccm_orange_rgb]
color = ['black', 'dgray', 'gray', 'dblue', 'blue', 'lblue', 'red', 'orange']

# Change decimal to binary
i = 0
for i in range(0, 8):
    color[i] = [colors[i] / 255 for colors[i] in colors[i]]

# Notification sounds
chime.theme('material')

### DATA PREPARATION

#### Load

In [4]:
# DATA PREPARATION
# Load file
directory = "C:/Users/André/Documents/00_ITA/00_Mestrado/20_Data_Preparation/"
file_name = "top_force.csv"

file = pd.read_csv(directory + file_name)
print("\n# File head\n", file.head())

# Dropping unnecessary columns
main_df = file.copy()
main_df.drop(['Exp', 'Tool', 'Block', 'SBlock',
           'Position', 'Condition', 'TCond', 'Run',
           'Length', 'Di', 'Df', 'CTime', 'RAngle'],
           axis=1, inplace=True)
print("\n\n\n# File head after dropping unnecessary columns\n", main_df.head())


# File head
    Exp  Tool  Block  SBlock Position  Condition  ...   RSm     Rt     Fx     Fy     Fz          F
0    0    21      1       2        a          4  ...  71.4  2.082  49.23  44.46  21.07  69.600499
1    0    21      1       2        a          4  ...  70.9  1.918  49.23  44.46  21.07  69.600499
2    0    21      1       2        a          4  ...  71.2  2.062  49.23  44.46  21.07  69.600499
3    0    21      1       2        a          4  ...  72.8  2.063  49.23  44.46  21.07  69.600499
4    0    21      1       2        a          4  ...  69.4  1.957  49.23  44.46  21.07  69.600499

[5 rows x 26 columns]



# File head after dropping unnecessary columns
      ap   vc     f     Ra     Rz    Rsk  ...   RSm     Rt     Fx     Fy     Fz          F
0  0.25  350  0.07  0.391  1.855  0.560  ...  71.4  2.082  49.23  44.46  21.07  69.600499
1  0.25  350  0.07  0.359  1.670  0.530  ...  70.9  1.918  49.23  44.46  21.07  69.600499
2  0.25  350  0.07  0.421  1.912  0.634  ...  71.2  2.

#### Split train and test sets

In [7]:
# Stratified train-test split (80/20)
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# split = ShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# split = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Stratified in relation to the feed rate ('f')
for train_index, test_index in split.split(main_df, main_df['f']):
    stratified_train = main_df.loc[train_index]
    stratified_test = main_df.loc[test_index]

# Check for stratification proportion correctness
print("\n# Percent of feed rate levels in the training set\n", stratified_train['f'].value_counts() / len(stratified_train))
print("\n# Percent of feed rate levels in the testing set\n", stratified_test['f'].value_counts() / len(stratified_test))

# Check for data leakage --- all ['Run'].value_counts must be == 6
print("\n# Number of Run ID values --- should be 6 to all", stratified_train['Run'].value_counts())

# Define features and labels
features = ['ap', 'vc', 'f', 'Fx', 'Fy', 'Fz', 'F']
labels = ['Ra']

x_train_and_validation = stratified_train.copy()[features]
x_test = stratified_test.copy()[features]

y_train = stratified_train.copy()[labels]
y_test = stratified_test.copy()[labels]

# Spare validation set
x_train, x_val,\
    y_train, y_val = train_test_split(x_train_and_validation, y_train,
    test_size = 0.2, random_state = 42)


# Percent of feed rate levels in the training set
 0.10    0.354167
0.07    0.333333
0.13    0.312500
Name: f, dtype: float64



# Percent of feed rate levels in the testing set
 0.07    0.416667
0.10    0.333333
0.13    0.250000
Name: f, dtype: float64

# Number of Run ID values 0_021_B1_4_a     6
0_021_B1_6_b     6
1_051_B1_5_a     6
1_051_B1_9_d     6
1_051_B1_8_e     6
1_051_B1_7_f     6
1_051_B1_2_g     6
1_051_B1_3_h     6
1_051_B1_1_i     6
1_051_B1_5_j     6
1_061_B2_11_a    6
1_061_B2_12_b    6
1_061_B2_10_c    6
1_061_B2_13_f    6
1_061_B2_16_g    6
1_061_B2_18_i    6
1_061_B2_11_j    6
1_071_B3_20_a    6
1_071_B3_21_c    6
1_071_B3_24_e    6
1_071_B3_23_f    6
1_071_B3_27_h    6
1_071_B3_26_i    6
0_041_B3_22_j    6
0_041_B3_20_i    6
0_041_B3_19_h    6
0_031_B2_18_g    6
0_021_B1_8_d     6
0_021_B1_9_e     6
0_021_B1_7_f     6
0_021_B1_2_g     6
0_021_B1_3_i     6
0_021_B1_4_j     6
0_031_B2_10_b    6
0_031_B2_17_e    6
0_031_B2_16_f    6
0_031_B2_14_h    6
0_041_B3_21_g  

#### Feature Scaling

In [14]:
# Scale x
# sc = preprocessing.StandardScaler(copy=True, with_std=True, with_mean=True)
sc_location = "C:/Users/André/Documents/00_ITA/00_Mestrado/70_pyproject/pyproject/"
sc_name = "standardize_all_data_wo_TCond.pkl"
sc = pickle.load(open(sc_location + sc_name, 'rb'))

#x_train_sc_np_array = sc.fit_transform(x_train)
x_train_sc_np_array = sc.transform(x_train)
x_train_sc = pd.DataFrame(data = x_train_sc_np_array,
    columns = x_train.columns, index = x_train.index)

x_val_sc_np_array = sc.transform(x_val)
x_val_sc = pd.DataFrame(data = x_val_sc_np_array,
    columns = x_val.columns, index=x_val.index)

x_test_sc_np_array = sc.transform(x_test)
x_test_sc = pd.DataFrame(data = x_test_sc_np_array, 
    columns = x_test.columns, index = x_test.index)

### MODEL BUILD

In [17]:
# MODEL BUILD
# Clean session
keras.backend.clear_session()

# Define standard layers
Regularized_Dense = partial(keras.layers.Dense, activation = "relu")


# Funtion to create model
def create_model():
    '''This function creates a sequential model'''
    model = keras.Sequential()
    model.add(keras.Input(shape = x_train_sc.shape[1:]))
    model.add(Regularized_Dense(28))
    # model.add(layers.Dropout(0.1))
    model.add(Regularized_Dense(56))
    # model.add(layers.Dropout(0.1))
    model.add(Regularized_Dense(14))
    # model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1))
    
    optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.95)
    
    model.compile(
        loss = "mean_squared_error",
        optimizer = optimizer,
        metrics = ["mean_absolute_percentage_error", "mean_absolute_error"])
    
    return model


es = EarlyStopping(monitor = "loss",
    min_delta = 0.0001,
    patience = 500,
    verbose = 1,
    mode = "min")

callbacks = [es]

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 28)                224       
_________________________________________________________________
dense_1 (Dense)              (None, 56)                1624      
_________________________________________________________________
dense_2 (Dense)              (None, 14)                798       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 15        
Total params: 2,661
Trainable params: 2,661
Non-trainable params: 0
_________________________________________________________________


### LEARN AND EVALUATE

In [16]:
# LEARN
epochs = 10000
history = model.fit(x_train_sc, y_train, epochs = epochs,
                    validation_data = (x_val_sc, y_val),
                    batch_size = 20, callbacks = callbacks,
                    verbose = 1)

# EVALUATE
loss = model.evaluate(x_test_sc, y_test)
print('\nModel evaluation: \n', loss)
%chime

Epoch 1/10000
12/12 [==============================] - 2s 28ms/step - loss: 0.1965 - mean_absolute_percentage_error: 56.5590 - mean_absolute_error: 0.3539 - val_loss: 0.0576 - val_mean_absolute_percentage_error: 38.9706 - val_mean_absolute_error: 0.1984
Epoch 2/10000
12/12 [==============================] - 0s 3ms/step - loss: 0.0698 - mean_absolute_percentage_error: 35.1177 - mean_absolute_error: 0.2060 - val_loss: 0.0348 - val_mean_absolute_percentage_error: 26.8218 - val_mean_absolute_error: 0.1406
Epoch 3/10000
12/12 [==============================] - 0s 3ms/step - loss: 0.0547 - mean_absolute_percentage_error: 35.1750 - mean_absolute_error: 0.1948 - val_loss: 0.0437 - val_mean_absolute_percentage_error: 29.7283 - val_mean_absolute_error: 0.1697
Epoch 4/10000
12/12 [==============================] - 0s 7ms/step - loss: 0.0415 - mean_absolute_percentage_error: 23.9953 - mean_absolute_error: 0.1584 - val_loss: 0.0409 - val_mean_absolute_percentage_error: 23.4363 - val_mean_absolute_e

## Using all data

### Feature selection & scaling

In [6]:
# Define features and labels
features = ['ap', 'vc', 'f', 'Fx', 'Fy', 'Fz', 'F']
labels = ['Ra']
x_main = main_df.copy()[features]
y = main_df.copy()[labels]

# Scale x
sc_location = "C:/Users/André/Documents/00_ITA/00_Mestrado/70_pyproject/pyproject/"
sc_name = "standardize_all_data_wo_TCond.pkl"
sc = pickle.load(open(sc_location + sc_name, 'rb'))

x_array = sc.transform(x_main)
x = pd.DataFrame(data = x_array,
    columns = x_main.columns, index = x_main.index)

In [7]:
# MODEL BUILD
# Clean session
keras.backend.clear_session()

# Define standard layers
Regularized_Dense = partial(keras.layers.Dense, activation = "relu")


# Funtion to create model
def create_model():
    '''This function creates a sequential model'''
    model = keras.Sequential()
    model.add(keras.Input(shape = x.shape[1:]))
    model.add(Regularized_Dense(28))
    # model.add(layers.Dropout(0.1))
    model.add(Regularized_Dense(56))
    # model.add(layers.Dropout(0.1))
    model.add(Regularized_Dense(14))
    # model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1))
    
    optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.95)
    
    model.compile(
        loss = "mean_squared_error",
        optimizer = optimizer,
        metrics = ["mean_absolute_percentage_error", "mean_absolute_error"])
    
    return model


es = EarlyStopping(monitor = "loss",
    min_delta = 0.0001,
    patience = 500,
    verbose = 1,
    mode = "min")

callbacks = [es]

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 28)                224       
_________________________________________________________________
dense_1 (Dense)              (None, 56)                1624      
_________________________________________________________________
dense_2 (Dense)              (None, 14)                798       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 15        
Total params: 2,661
Trainable params: 2,661
Non-trainable params: 0
_________________________________________________________________


In [8]:
# LEARN
epochs = 10000
history = model.fit(x, y, epochs = epochs,
                    batch_size = 20, callbacks = callbacks,
                    verbose = 1)

# EVALUATE
loss = model.evaluate(x, y)
print('\nModel evaluation: \n', loss)
%chime

Epoch 1/10000
